# Autoencoding beyond pixels using a learned similarity metric
* paper: [Autoencoding beyond pixels using a learned similarity metric](https://arxiv.org/pdf/1512.09300.pdf)

## Implementation Details
* **Training Data**: `FGVC-Aircraft`
* **Goal**: Generate Random New Images of Aircraft
* **Encoder Layers**: *In progress*
* **Decoder Layers**: *In progress*

## Notes
* using the results of this to compare against results from basic implementation of vanilla [VAE](https://github.com/bradley-ray/implementing-deep-learn-papers/blob/master/variational_autoencoder.ipynb)